### **Programming in Data Science**
### **Project n° 5 - Climate Watch historical emissions data**

#### Group members : 
- Anais Bellagha 
- Laura Deleuze
- Laura Aboukrat
- Lilian Allio
- Brunthaban Biradepan
- Eliott Alfandari

### Part 0 - Importation and exploration 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from statsmodels.tsa.arima.model import ARIMA

import plotly.express as px

In [ ]:
def load_data(file_path):
    """
    Input: file_path (str) – path to the CSV file
    Output: df (DataFrame) – loaded dataset
    """
    df = pd.read_csv(file_path)
    return df


def explore_data(df):
    """
    Prints structural information
    and generates basic exploratory visualizations.
    """
    print("Dataset Shape:", df.shape)
    print("\nColumn Types:\n", df.dtypes)

    print("\nMissing Values per Column:\n", df.isna().sum())

    print("\nStatistical Summary:\n", df.describe())

    numeric_cols = df.select_dtypes(include=[np.number]).columns
    if len(numeric_cols) > 1:
        plt.figure(figsize=(10, 6))
        sns.heatmap(df[numeric_cols].corr(), annot=False, cmap="coolwarm")
        plt.title("Correlation Heatmap")
        plt.show()
    else:
        print("\nNot enough numeric columns to compute correlations.")

In this step, we explore the structure of the dataset to understand its content. We examine:

- number of rows and columns,
- data types,
- missing values,
- numerical statistics,
- correlations between variables,
- basic visualizations (heatmap).

This information helps identify potential issues and guides the construction of the indicators in later steps.

### Part 1 - Indicators / Group By

#### Top N emitting countries in a specific year

In [3]:
def indicator_1a_top_emitters_2021(df, year=2021, top_n=10, color="orange", return_figure=False):
    """
    Indicator 1A:
        Top N emitting countries in a specific year.
    
    Parameters:
        df (DataFrame): Emissions dataset
        year (int): Year to analyze (default: 2021)
        top_n (int): Number of top countries to display (default: 10)
        color (str): Bar color for the plot (default: "orange")
        return_figure (bool): If True, return plotly figure instead of showing (default: False)
    """
    year_str = str(year)
    df_temp = df.copy()
    df_temp = df_temp[
        (df_temp["Country"] != "World") &
        (df_temp["Country"] != "European Union (27)")
    ]
    top_countries = df_temp.groupby("Country")[year_str].sum().sort_values(ascending=False).head(top_n)
    
    if return_figure:
        import plotly.graph_objects as go
        fig = go.Figure(data=[go.Bar(x=top_countries.index, y=top_countries.values, marker_color=color)])
        fig.update_layout(
            title=f"Top {top_n} Emitting Countries ({year})",
            xaxis_title="Country",
            yaxis_title="Emissions",
            height=450
        )
        return fig
    
    print(f"Top {top_n} emitting countries ({year}):")
    print(top_countries)

    plt.figure(figsize=(10,6))
    top_countries.plot(kind="bar", color=color)
    plt.title(f"Top {top_n} Emitting Countries ({year})")
    plt.ylabel("Emissions")
    plt.xlabel("Country")
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()
    plt.show()

#### Top N countries with the largest increase in emissions between two years

In [4]:
def indicator_1b_largest_increase(df, start_year=1990, end_year=2021, top_n=10, color="red"):
    """
    Indicator 1B:
        Top N countries with the largest increase in emissions between two years.
    
    Parameters:
        df (DataFrame): Emissions dataset
        start_year (int): Starting year for comparison (default: 1990)
        end_year (int): Ending year for comparison (default: 2021)
        top_n (int): Number of top countries to display (default: 10)
        color (str): Bar color for the plot (default: "red")
    """
    start_str = str(start_year)
    end_str = str(end_year)
    
    temp = df[["Country", start_str, end_str]].copy()
    temp[[start_str, end_str]] = temp[[start_str, end_str]].fillna(0)
    temp[f"Change_{start_year}_{end_year}"] = temp[end_str] - temp[start_str]

    top_increase = temp.groupby("Country")[f"Change_{start_year}_{end_year}"].sum().sort_values(ascending=False).head(top_n)
    print(f"\nTop {top_n} countries with the largest increase in emissions ({start_year}–{end_year}):")
    print(top_increase)

    plt.figure(figsize=(10,6))
    top_increase.plot(kind="bar", color=color)
    plt.title(f"Top {top_n} Countries – Increase in Emissions ({start_year}–{end_year})")
    plt.ylabel(f"Increase in Emissions ({end_year} – {start_year})")
    plt.xlabel("Country")
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()
    plt.show()

#### Top N lowest emitting countries in a specific year

In [5]:
def indicator_1c_lowest_emitters_2021(df, year=2021, top_n=10, color="green"):
    """
    Indicator 1C:
        Top N lowest emitting countries in a specific year.
    
    Parameters:
        df (DataFrame): Emissions dataset
        year (int): Year to analyze (default: 2021)
        top_n (int): Number of lowest countries to display (default: 10)
        color (str): Bar color for the plot (default: "green")
    """
    year_str = str(year)
    lowest_countries = df.groupby("Country")[year_str].sum().sort_values(ascending=True).head(top_n)

    print(f"Top {top_n} lowest emitting countries ({year}):")
    print(lowest_countries)

    plt.figure(figsize=(10,6))
    lowest_countries.plot(kind="bar", color=color)
    plt.title(f"Top {top_n} Lowest Emitting Countries ({year})")
    plt.ylabel("Emissions")
    plt.xlabel("Country")
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()
    plt.show()

#### Total global emissions per year over a time period

In [6]:
def indicator_1d_global_emissions_trend(df, start_year=1990, end_year=2021, color="blue"):
    """
    Indicator 1D:
        Total global emissions per year over a time period.
    
    Parameters:
        df (DataFrame): Emissions dataset
        start_year (int): Starting year for the trend (default: 1990)
        end_year (int): Ending year for the trend (default: 2021)
        color (str): Line color for the plot (default: "blue")
    """
    years = [str(y) for y in range(start_year, end_year + 1)]
    available_years = [y for y in years if y in df.columns]

    total_per_year = df[available_years].sum()

    print(f"\nTotal global emissions per year ({start_year}–{end_year}):")
    print(total_per_year)

    plt.figure(figsize=(12,6))
    total_per_year.plot(kind="line", marker="o", color=color)
    plt.title(f"Total Global Emissions per Year ({start_year}–{end_year})")
    plt.ylabel("Emissions")
    plt.xlabel("Year")
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.tight_layout()
    plt.show()

#### Emissions trend for a specific country over time

In [7]:
def indicator_1e_emissions(df, country="France", start_year=1990, end_year=2021, color="red", return_figure=False):
    """
    Indicator 1E:
        Emissions trend for a specific country over time.
    
    Parameters:
        df (DataFrame): Emissions dataset
        country (str): Country name to analyze (default: "China")
        start_year (int): Starting year for the trend (default: 1990)
        end_year (int): Ending year for the trend (default: 2021)
        color (str): Line color for the plot (default: "red")
        return_figure (bool): If True, return plotly figure instead of showing (default: False)
    """
    years = [str(y) for y in range(start_year, end_year + 1)]
    available_years = [y for y in years if y in df.columns]
    
    country_data = df[df["Country"] == country]
    
    if country_data.empty:
        if return_figure:
            import plotly.graph_objects as go
            fig = go.Figure()
            fig.add_annotation(text=f"Country '{country}' not found", showarrow=False)
            return fig
        print(f"Warning: Country '{country}' not found in the dataset.")
        return
    
    country_emissions = country_data[available_years].sum()
    
    if return_figure:
        import plotly.graph_objects as go
        fig = go.Figure(data=[go.Scatter(
            x=available_years, y=country_emissions.values, mode='lines+markers',
            fill='tozeroy',
            fillcolor=f'rgba({int(color[1:3], 16) if color.startswith("#") else 70}, {int(color[3:5], 16) if color.startswith("#") else 130}, {int(color[5:7], 16) if color.startswith("#") else 180}, 0.3)' if color.startswith('#') else 'rgba(255, 0, 0, 0.3)',
            line=dict(color=color, width=2), marker=dict(size=5, color=color)
        )])
        fig.update_layout(
            title=f"{country}'s Emissions ({start_year}–{end_year})",
            xaxis_title="Year",
            yaxis_title="Emissions",
            height=450
        )
        return fig

    plt.figure(figsize=(12,6))
    country_emissions.plot(kind="line", marker="o", color=color)
    plt.title(f"{country}'s Emissions ({start_year}–{end_year})")
    plt.xlabel("Year")
    plt.ylabel("Emissions")
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.tight_layout()
    plt.show()

#### Emissions comparison for multiple countries/regions over time

In [8]:
def indicator_1f_countries_comparison(df, countries=None, start_year=1990, end_year=2021, 
                                       colors=None, figsize=(14,7), return_figure=False):
    """
    Indicator 1G:
        Emissions comparison for multiple countries/regions over time.
    
    Parameters:
        df (DataFrame): Emissions dataset
        countries (list): List of country names to compare 
                         (default: ["China", "France", "United States", "India", "European Union"])
        start_year (int): Starting year for comparison (default: 1990)
        end_year (int): Ending year for comparison (default: 2021)
        colors (list): List of colors for each country (default: predefined colors)
        figsize (tuple): Figure size (default: (14,7))
        return_figure (bool): If True, return plotly figure instead of showing (default: False)
    """
    if countries is None:
        countries = ["China", "France", "United States", "India", "European Union"]
    
    if colors is None:
        colors = ["red", "green", "orange", "purple", "brown"]
    
    # Ensure we have enough colors
    if len(colors) < len(countries):
        colors = colors + ["gray"] * (len(countries) - len(colors))
    
    years = [str(y) for y in range(start_year, end_year + 1)]
    available_years = [y for y in years if y in df.columns]
    
    if return_figure:
        import plotly.graph_objects as go
        fig = go.Figure()
        for idx, country in enumerate(countries):
            country_data = df[df["Country"] == country]
            if country_data.empty:
                continue
            country_emissions = country_data[available_years].sum()
            fig.add_trace(go.Scatter(
                x=available_years, y=country_emissions.values,
                mode='lines+markers', name=country,
                line=dict(color=colors[idx % len(colors)], width=2),
                marker=dict(size=5)
            ))
        fig.update_layout(
            title="Multi-Country Emissions Comparison",
            xaxis_title="Year",
            yaxis_title="Emissions",
            height=450,
            legend=dict(orientation="v", yanchor="top", y=1, xanchor="left", x=1.02)
        )
        return fig
    
    plt.figure(figsize=figsize)

    for idx, country in enumerate(countries):
        country_data = df[df["Country"] == country]
        
        if country_data.empty:
            print(f"Warning: Country '{country}' not found in the dataset.")
            continue
        
        country_emissions = country_data[available_years].sum()
        plt.plot(available_years, country_emissions, marker="o", 
                label=country, color=colors[idx])

    plt.title(f"Emissions Comparison: {', '.join(countries)} ({start_year}–{end_year})")
    plt.xlabel("Year")
    plt.ylabel("Emissions")
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()

#### Comments

- These functions are designed to explore and visualize overall greenhouse gas emission trends. For this, we use only the columns containing the country name and the years from 1990 to 2021 are used, as they are the only variables consistent and comparable across the entire dataset. Other columns (such as Data source, Gas, Sector, Unit) are not used, as they are unique to each row, which prevents any meaningful global aggregation or comparison.

- Indicator 1A is the top 10 emitting countries in 2021. This indicator calculates, for each country, the total sum of its emissions in 2021. We can identifie the 10 countries that emitted the most that year.

- Indicator 1B correpond to the largest increase in emissions (1990–2021).this indicator compares each country's emissions in 1990 and 2021 to measure the total change over this period. It highlights the 10 countries whose emissions increased the most, allowing for the identification of the largest growths.

- Indicator 1C show the lowest emitting countries in 2021. Here, the function identifies the 10 countries with the lowest emissions in 2021 by aggregating the values by country. This highlights the countries that contribute the least to global emissions

- Indicator 1D display the total global emissions per year (1990–2021).this indicator sums up the global emissions for each year between 1990 and 2021. It allows us to visualize the overall evolution of emissions on a global scale. We therefore have a time curve that clearly shows the trends.

- Indicator 1E permite to see France's emissions trend (1990–2021). it extracts all the rows corresponding to France and sums the annual values to obtain the time series of Chinese emissions over 32 years. The indicator then plots a curve showing the country's emissions evolution year by year. This allows us to observe the dynamics of a major global emitter

- Indicator 1G is a comparison between China, France, US, India, and the EU
This indicator selects and aggregates the annual values of five major regions/countries. It then traces the evolution of each one's emissions on the same chart to compare their respective trajectories. This visualization highlights differences in emission levels, dynamics, and transitions among these key actors.


### Part 2 - Indicators / Transformation

In [9]:
def indicator_transformation(df):
    """
    Indicator 2A:
        Standardize country emission time series (1990–2021) using Z-score.

    Indicator 2B:
        K-Means clustering of countries based on standardized emission trends.

    Indicator 2C:
        PCA projection of clusters for visualization in 2D.
    """

    # Prepare the data
    temp = df.copy()
    years = [str(y) for y in range(1990, 2022)]
    country_data = temp.groupby("Country")[years].sum()

    # Indicator 2A: Standardize (Z-score)
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(country_data)

    print("Indicator 2A – Standardized emissions (first 5 countries):")
    print(pd.DataFrame(X_scaled, index=country_data.index, columns=years).head())

    # Indicator 2B:K-Means Clustering
    n_clusters = 3  # hardcoded number of clusters
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    clusters = kmeans.fit_predict(X_scaled)

    country_data["Cluster"] = clusters
    print(f"\nIndicator 2B – K-Means clustering results (first 10 countries):")
    print(country_data.head(10))

    # Indicator 2C: PCA Projection for visualization
    pca = PCA(n_components=2)
    proj = pca.fit_transform(X_scaled)

    plt.figure(figsize=(12, 7))
    scatter = plt.scatter(proj[:, 0], proj[:, 1], c=clusters, cmap="viridis", s=100)
    for i, country in enumerate(country_data.index):
        plt.text(proj[i, 0]+0.1, proj[i, 1]+0.1, country, fontsize=8)
    plt.title(f"Indicator 2C – K-Means Clustering of Countries (PCA 2D, {n_clusters} clusters)")
    plt.xlabel("PC1")
    plt.ylabel("PC2")
    plt.colorbar(scatter, label="Cluster")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    return country_data

- Indicator 2A is the standardization of time series (Z-score) This indicator transforms each country's emissions to make them comparable to one another. Standardization (Z-score) recenters the data around 0 and scales it to one standard deviation, which explains why values often become negative or close to 0. This step allows for analyzing only the shape of trends (increase, decrease, stability) without being influenced by countries that emit a lot (e.g., China, USA).

- Indicator 2B is the K-Means clustering of countries based on their emission profiles
This indicator applies the K-Means algorithm to group countries into three clusters based on their standardized emission trends. Countries are not grouped by emission intensity, but rather by similarity in trends: rapid growth, stability, or gradual decline. This creates groups of countries that share similar behaviors over the past 30 years

- Indicator 2C permite to visualize clusters via PCA (2D projection)This indicator uses PCA to reduce the 32 years of data to just two dimensions, in order to visualize the clusters on a 2D plane. PCA retains most of the data variance, which allows for an effective representation of the groupings created by K-Means. The final graph shows countries as points colored according to their cluster, making it easier to visually interpret the different emission behaviors.

### Part 3 - Indicators / Temporal

#### Indicator 3A — ARIMA Forecast of Global Emissions

In this indicator, we construct a global time series by summing emissions from all individual countries (excluding aggregate entries such as "World" or "European Union (27)").
We then apply an ARIMA (1,1,1) model to the historical emissions from 1990 to 2021 to forecast the next ten years.

The resulting plot includes:

* the historical emissions curve,
* the ARIMA forecast (dashed line),
* a 95% confidence interval.

This indicator provides a data-driven projection of global emissions if past long-term dynamics continue. It also illustrates the uncertainty inherent in forecasting environmental variables over time.

In [10]:
def indicator_3a_arima_forecast(df, start_year=1990, end_year=2021, forecast_steps=10, 
                                 order=(1, 1, 1), figsize=(10, 6), 
                                 exclude_aggregates=True, return_figure=False):
    """
    Indicator 3A:
        ARIMA forecast of global emissions.
    
    Parameters:
        df (DataFrame): Emissions dataset
        start_year (int): Starting year for historical data (default: 1990)
        end_year (int): Ending year for historical data (default: 2021)
        forecast_steps (int): Number of years to forecast (default: 10)
        order (tuple): ARIMA model order (p, d, q) (default: (1, 1, 1))
        figsize (tuple): Figure size (default: (10, 6))
        exclude_aggregates (bool): Exclude "World" and aggregate entries (default: True)
        return_figure (bool): If True, return plotly figure instead of showing (default: False)
    
    Returns:
        dict or figure: Dictionary with forecast results or plotly figure
    """
    df_countries = df.copy()
    
    if exclude_aggregates:
        df_countries = df_countries[
            (df_countries["Country"] != "World") &
            (df_countries["Country"] != "European Union (27)")
        ]
    
    year_cols = [str(y) for y in range(start_year, end_year + 1)]
    available_years = [y for y in year_cols if y in df_countries.columns]
    
    ts = df_countries[available_years].sum(axis=0)
    
    ts.index = ts.index.astype(int)
    ts = ts.sort_index()
    
    ts = ts.fillna(method="ffill").fillna(method="bfill")
    
    model = ARIMA(ts, order=order)
    results = model.fit()
    
    forecast = results.get_forecast(steps=forecast_steps)
    forecast_mean = forecast.predicted_mean
    forecast_ci = forecast.conf_int()
    
    last_year = ts.index.max()
    future_years = np.arange(last_year + 1, last_year + 1 + forecast_steps)
    
    if return_figure:
        import plotly.graph_objects as go
        fig = go.Figure()
        
        fig.add_trace(go.Scatter(
            x=ts.index, y=ts.values,
            mode='lines+markers',
            name='Historical',
            line=dict(color='blue', width=2)
        ))
        
        fig.add_trace(go.Scatter(
            x=future_years, y=forecast_mean.values,
            mode='lines+markers',
            name='ARIMA Forecast',
            line=dict(color='red', width=2, dash='dash')
        ))
        
        fig.add_trace(go.Scatter(
            x=np.concatenate([future_years, future_years[::-1]]),
            y=np.concatenate([forecast_ci.iloc[:, 1], forecast_ci.iloc[:, 0][::-1]]),
            fill='toself',
            fillcolor='rgba(255,0,0,0.2)',
            line=dict(color='rgba(255,255,255,0)'),
            showlegend=True,
            name='95% Confidence'
        ))
        
        fig.update_layout(
            title=f"ARIMA Forecast - Global Emissions ({forecast_steps} years ahead)",
            xaxis_title="Year",
            yaxis_title="Emissions",
            height=450
        )
        return fig
    
    print(f"Global emissions time series ({start_year}–{end_year}):")
    print(f"First values:\n{ts.head()}")
    print(f"\nLast values:\n{ts.tail()}")
    
    plt.figure(figsize=figsize)
    plt.plot(ts.index, ts.values, label="Historical emissions", marker="o")
    plt.plot(future_years, forecast_mean.values,
             label=f"ARIMA{order} forecast", linestyle="--", marker="o")
    
    plt.fill_between(
        future_years,
        forecast_ci.iloc[:, 0],
        forecast_ci.iloc[:, 1],
        alpha=0.2,
        label="95% confidence interval"
    )
    
    plt.title(f"Indicator 3A – ARIMA Forecast of Global Emissions ({start_year}–{end_year + forecast_steps})")
    plt.xlabel("Year")
    plt.ylabel("Emissions")
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    return {
        'model': results,
        'time_series': ts,
        'forecast_mean': forecast_mean,
        'confidence_interval': forecast_ci,
        'future_years': future_years
    }

#### Indicator 3B — Linear Trend and Forecast of Global Emissions

In parallel with ARIMA, we fit a simple linear regression model to the same global emission time series.
This model captures the long-term linear trend between emissions and time and is then used to predict emissions up to 2030.

The plot shows:

* historical emissions (scatter points),
* the fitted linear trend line,
* future predicted emissions (dashed line).

This indicator offers a simplified but intuitive forecast of emissions. Comparing it with the ARIMA forecast helps illustrate differences between linear and statistical time-series models.

In [11]:
def indicator_3b_linear_regression_forecast(df, start_year=1990, end_year=2021, 
                                             forecast_to_year=2030, figsize=(10, 6),
                                             exclude_aggregates=True, return_figure=False):
    """
    Indicator 3B:
        Linear regression trend and forecast of global emissions.
    
    Parameters:
        df (DataFrame): Emissions dataset
        start_year (int): Starting year for historical data (default: 1990)
        end_year (int): Ending year for historical data (default: 2021)
        forecast_to_year (int): Year to forecast until (default: 2030)
        figsize (tuple): Figure size (default: (10, 6))
        exclude_aggregates (bool): Exclude "World" and aggregate entries (default: True)
        return_figure (bool): If True, return plotly figure instead of showing (default: False)
    
    Returns:
        dict or figure: Dictionary with regression results or plotly figure
    """
    df_countries = df.copy()
    
    if exclude_aggregates:
        df_countries = df_countries[
            (df_countries["Country"] != "World") &
            (df_countries["Country"] != "European Union (27)")
        ]
    
    year_cols = [str(y) for y in range(start_year, end_year + 1)]
    available_years = [y for y in year_cols if y in df_countries.columns]
    
    ts = df_countries[available_years].sum(axis=0)
    
    ts.index = ts.index.astype(int)
    ts = ts.sort_index()
    
    ts = ts.fillna(method="ffill").fillna(method="bfill")
    
    X = ts.index.values.reshape(-1, 1)  
    y = ts.values  
    
    lin_reg = LinearRegression()
    lin_reg.fit(X, y)
    
    r_squared = lin_reg.score(X, y)
    
    years_full = np.arange(start_year, forecast_to_year + 1)
    y_pred = lin_reg.predict(years_full.reshape(-1, 1))
    
    if return_figure:
        import plotly.graph_objects as go
        fig = go.Figure()
        
        fig.add_trace(go.Scatter(
            x=ts.index, y=ts.values,
            mode='markers',
            name='Historical',
            marker=dict(color='blue', size=8)
        ))
        
        fig.add_trace(go.Scatter(
            x=years_full, y=y_pred,
            mode='lines',
            name=f'Linear Trend (R²={r_squared:.3f})',
            line=dict(color='red', width=2)
        ))
        
        future_mask = years_full > end_year
        fig.add_trace(go.Scatter(
            x=years_full[future_mask], y=y_pred[future_mask],
            mode='lines',
            name='Forecast',
            line=dict(color='orange', width=2, dash='dash')
        ))
        
        fig.update_layout(
            title=f"Linear Trend & Forecast - Global Emissions (to {forecast_to_year})",
            xaxis_title="Year",
            yaxis_title="Emissions",
            height=450
        )
        return fig
    
    plt.figure(figsize=figsize)
    plt.scatter(ts.index, ts.values,
                label="Historical emissions", s=50, alpha=0.7)
    plt.plot(years_full, y_pred,
             label=f"Linear regression (R²={r_squared:.3f})", 
             linestyle="-", color="red", linewidth=2)
    
    future_mask = years_full > end_year
    plt.plot(years_full[future_mask], y_pred[future_mask],
             label="Forecast", linestyle="--", color="orange", linewidth=2)
    
    plt.title(f"Indicator 3B – Linear Trend and Forecast ({start_year}–{forecast_to_year})")
    plt.xlabel("Year")
    plt.ylabel("Emissions")
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    print(f"\nLinear Regression Results:")
    print(f"  Slope: {lin_reg.coef_[0]:.2f} (emissions change per year)")
    print(f"  Intercept: {lin_reg.intercept_:.2f}")
    print(f"  R² Score: {r_squared:.4f}")
    
    return {
        'model': lin_reg,
        'time_series': ts,
        'years_full': years_full,
        'predictions': y_pred,
        'r_squared': r_squared,
        'slope': lin_reg.coef_[0],
        'intercept': lin_reg.intercept_
    }

#### Indicator 3C — Comparison of Emission Trends for Selected Countries

To complement the global analysis, we compare the emission trajectories of several major countries over the same period (1990–2021).
For each selected ISO code, we extract the country’s time series and plot them together on a single graph.

This comparison reveals meaningful differences between countries:

* some exhibit fast-growing emissions,
* others show moderate growth or stabilization,
* a few display declining emissions over time.

This indicator provides important context by showing how national trajectories differ from the global trend and from one another.

In [12]:
def indicator_3c_countries_comparison(df, countries=None, iso_codes=None, 
                                       start_year=1990, end_year=2021, 
                                       colors=None, figsize=(10, 6),
                                       use_iso=False):
    """
    Indicator 3C:
        Comparison of emission trends for selected countries.
    
    Parameters:
        df (DataFrame): Emissions dataset
        countries (list): List of country names to compare 
                         (default: ["China", "United States", "India", "France"])
        iso_codes (list): List of ISO codes to compare (alternative to countries)
                         (default: ["CHN", "USA", "IND", "FRA"])
        start_year (int): Starting year for comparison (default: 1990)
        end_year (int): Ending year for comparison (default: 2021)
        colors (list): List of colors for each country (default: predefined)
        figsize (tuple): Figure size (default: (10, 6))
        use_iso (bool): Use ISO codes instead of country names (default: False)
    
    Returns:
        dict: Dictionary with country time series data
    """
    if use_iso or (iso_codes is not None and countries is None):
        use_iso = True
        if iso_codes is None:
            iso_codes = ["CHN", "USA", "IND", "FRA"]
        identifiers = iso_codes
        id_column = "ISO"
    else:
        if countries is None:
            countries = ["China", "United States", "India", "France"]
        identifiers = countries
        id_column = "Country"
    
    if colors is None:
        colors = ["red", "blue", "green", "orange", "purple", "brown", "pink", "gray"]
    
    if len(colors) < len(identifiers):
        colors = colors + ["gray"] * (len(identifiers) - len(colors))
    
    df_clean = df.copy()
    df_clean["ISO"] = df_clean["ISO"].str.strip()
    df_clean["Country"] = df_clean["Country"].str.strip()
    
    years = [str(y) for y in range(start_year, end_year + 1)]
    available_years = [y for y in years if y in df_clean.columns]
    
    plt.figure(figsize=figsize)
    
    country_data = {}
    
    for idx, identifier in enumerate(identifiers):
        country_df = df_clean[df_clean[id_column] == identifier]
        
        if country_df.empty:
            print(f"Warning: {id_column} '{identifier}' not found in the dataset.")
            continue
        
        ts = country_df[available_years].sum().values
        country_name = country_df["Country"].iloc[0]
        
        plt.plot(available_years, ts, marker="o", label=country_name, 
                color=colors[idx], linewidth=2)
        
        country_data[country_name] = {
            'years': available_years,
            'emissions': ts,
            'identifier': identifier
        }
    
    plt.title(f"Indicator 3C – Comparison of Emissions for Selected Countries ({start_year}–{end_year})")
    plt.xlabel("Year")
    plt.ylabel("Emissions")
    plt.xticks(rotation=45)
    plt.legend(loc='best')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    return country_data

### Part 4 - Indicators / Spatial

#### Indicator 4A – Share of global emissions by country

Instead of plotting raw emissions, we compute for each country its **share of global emissions in 2021 (for example)**.
We first aggregate emissions per country and then divide by the world total to obtain a percentage.
The choropleth map highlights:

* a small number of countries concentrating a very large share of global emissions,
* many countries with a very small contribution in absolute terms.

This indicator is more informative than raw values, as it directly answers the question:
**“Which countries contribute the most to global emissions?”**

In [ ]:
def indicator_4a_emission_share(df, year=2021, color_scale="Reds", 
                                 exclude_aggregates=True, show_top_n=None, show_plot=True):
    """
    Indicator 4A:
        Share of global emissions (%) by country for a specific year.
    
    Parameters:
        df (DataFrame): Emissions dataset
        year (int): Year to analyze (default: 2021)
        color_scale (str): Color scale for the choropleth map (default: "Reds")
        exclude_aggregates (bool): Exclude "World" and aggregate entries (default: True)
        show_top_n (int): If set, print the top N countries by share (default: None)
        show_plot (bool): If True, display the choropleth map (default: True)
    
    Returns:
        DataFrame: Country emissions and their share of global total
    """
    df_countries = df.copy()
    df_countries["ISO"] = df_countries["ISO"].str.strip()
    df_countries["Country"] = df_countries["Country"].str.strip()

    if exclude_aggregates:
        df_countries = df_countries[
            (df_countries["Country"] != "World") &
            (df_countries["Country"] != "European Union (27)")
        ]

    year_str = str(year)
    emissions_year = (
        df_countries.groupby(["ISO", "Country"])[year_str]
        .sum()
        .reset_index()
    )

    total_year = emissions_year[year_str].sum()
    emissions_year[f"Share_{year}"] = emissions_year[year_str] / total_year * 100

    if show_top_n:
        top_countries = emissions_year.nlargest(show_top_n, f"Share_{year}")
        print(f"\nTop {show_top_n} countries by share of global emissions ({year}):")
        for idx, row in top_countries.iterrows():
            print(f"  {row['Country']}: {row[f'Share_{year}']:.2f}%")

    if show_plot:
        fig_share = px.choropleth(
            emissions_year,
            locations="ISO",
            color=f"Share_{year}",
            hover_name="Country",
            hover_data={year_str: True, f"Share_{year}": ":.2f"},
            color_continuous_scale=color_scale,
            labels={f"Share_{year}": f"Share of global emissions (%)"},
            title=f"Indicator 4A – Share of Global Emissions by Country ({year})"
        )
        fig_share.show()

    return emissions_year

#### Indicator 4B — Emission Trend Slope (1990–2021)

For each country, we compute a linear regression of annual greenhouse gas emissions from 1990 to 2021.

**Formula** : Emissions = *a* x Year + *b*

**Interpretation of the slope (the coefficient "a"):**

* **Slope > 0** → emissions are increasing
* **Slope < 0** → emissions are decreasing
* **Slope ≈ 0** → emissions are relatively stable

This indicator highlights long-term emission dynamics at the country level.
It reveals three important patterns:

1. **Fast-growing emitters** (large positive slopes), often rapidly industrializing economies
2. **Stabilizing or slowly increasing countries**
3. **Countries with declining emissions** (negative slopes), usually due to climate policies or economic restructuring

This spatial visualization complements the global ARIMA and linear regression indicators by showing **where** emission trajectories are rising or falling across the world.

In [ ]:
def indicator_4b_emission_trend_slope(df, start_year=1990, end_year=2021, 
                                       color_scale=None, clip_quantiles=(0.05, 0.95),
                                       exclude_aggregates=True, show_top_n=None,
                                       show_bottom_n=None):
    """
    Indicator 4B:
        Emission trend slope by country between two years using linear regression.
    
    Parameters:
        df (DataFrame): Emissions dataset
        start_year (int): Starting year for trend analysis (default: 1990)
        end_year (int): Ending year for trend analysis (default: 2021)
        color_scale (list): Custom color scale for diverging map (default: blue-white-red)
        clip_quantiles (tuple): Quantiles to clip extreme values (default: (0.05, 0.95))
        exclude_aggregates (bool): Exclude "World" and aggregate entries (default: True)
        show_top_n (int): If set, print top N countries with highest slopes (default: None)
        show_bottom_n (int): If set, print bottom N countries with lowest slopes (default: None)
    
    Returns:
        DataFrame: Country slopes and trend information
    """
    if color_scale is None:
        color_scale = [
            (0.00, "#2B1F5C"), 
            (0.25, "#4B74C8"),  
            (0.50, "#E8E8E8"),  
            (0.75, "#F97B22"),  
            (1.00, "#7A0403")   
        ]

    df_temp = df.copy()
    df_temp["ISO"] = df_temp["ISO"].str.strip()
    df_temp["Country"] = df_temp["Country"].str.strip()

    if exclude_aggregates:
        df_temp = df_temp[
            (df_temp["Country"] != "World") &
            (df_temp["Country"] != "European Union (27)")
        ]

    slopes = []
    years = [str(y) for y in range(start_year, end_year + 1)]
    X = np.array([int(y) for y in years]).reshape(-1, 1)

    for iso, g in df_temp.groupby("ISO"):
        y_vals = g[years].sum().values

        if np.all(np.isnan(y_vals)):
            continue

        y_vals = np.nan_to_num(y_vals)

        model = LinearRegression().fit(X, y_vals)
        slope = model.coef_[0]

        slopes.append({
            "ISO": iso,
            "Country": g["Country"].iloc[0],
            "Slope": slope
        })

    slopes_df = pd.DataFrame(slopes)
    
    slopes_df["Slope_clipped"] = slopes_df["Slope"].clip(
        lower=slopes_df["Slope"].quantile(clip_quantiles[0]),
        upper=slopes_df["Slope"].quantile(clip_quantiles[1])
    )

    if show_top_n:
        top_slopes = slopes_df.nlargest(show_top_n, "Slope")
        print(f"\nTop {show_top_n} countries with highest emission growth ({start_year}–{end_year}):")
        for idx, row in top_slopes.iterrows():
            print(f"  {row['Country']}: {row['Slope']:.2f} (slope)")
    
    if show_bottom_n:
        bottom_slopes = slopes_df.nsmallest(show_bottom_n, "Slope")
        print(f"\nTop {show_bottom_n} countries with lowest/declining emissions ({start_year}–{end_year}):")
        for idx, row in bottom_slopes.iterrows():
            print(f"  {row['Country']}: {row['Slope']:.2f} (slope)")

    fig_slope = px.choropleth(
        slopes_df,
        locations="ISO",
        color="Slope_clipped",
        hover_name="Country",
        hover_data={"Slope": ":.2f"},
        color_continuous_scale=color_scale,
        color_continuous_midpoint=0,
        labels={"Slope_clipped": f"Trend slope ({start_year}–{end_year})"},
        title=f"Indicator 4B – Emission Trend Slope by Country ({start_year}–{end_year})"
    )

    fig_slope.update_layout(
        coloraxis_colorbar=dict(
            title="Slope",
            tickformat=".2f"
        )
    )

    fig_slope.show()

    return slopes_df

### Part 5 - Dashboard for vizualisations

In [ ]:
from dash import Dash, dcc, html, Input, Output
import dash_bootstrap_components as dbc

def create_dashboard(df):
    """
    Creates and launches the Dash dashboard using existing indicator functions.
    """
    years = list(range(1990, 2022))
    df_temp = df.copy()
    df_temp = df_temp[
        (df_temp["Country"] != "World") &
        (df_temp["Country"] != "European Union (27)")
    ]
    countries = sorted(df_temp["Country"].unique())
    
    app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
    server = app.server
    
    navbar_filters = dbc.Navbar(
        children=dbc.Container(
            dbc.Row(
                [
                    dbc.Col(
                        [
                            html.Label(
                                "Year",
                                className="fw-bold",
                                style={"font-size": "16px"}
                            ),
                            dcc.Slider(
                                id="year-slider",
                                min=min(years),
                                max=max(years),
                                value=max(years),
                                marks={str(y): str(y) if y % 10 == 0 else "" for y in years},
                                step=1,
                            ),
                        ],
                        width=2,
                        style={"padding": "0 10px"},
                    ),

                    dbc.Col(
                        [
                            html.Label(
                                "Top N",
                                className="fw-bold",
                                style={"font-size": "16px"}
                            ),
                            dcc.RadioItems(
                                id="top-n-radio",
                                options=[{"label": str(k), "value": k} for k in [5, 10, 15, 20]],
                                value=10,
                                inline=True,
                                style={"font-size": "15px"},
                            ),
                        ],
                        width=1,
                        style={"padding": "0 10px"},
                    ),

                    dbc.Col(
                        [
                            html.Label(
                                "Country",
                                className="fw-bold",
                                style={"font-size": "16px"}
                            ),
                            dcc.Dropdown(
                                id="country-single",
                                options=[{"label": c, "value": c} for c in countries],
                                value=countries[0] if countries else None,
                                clearable=False,
                                style={"font-size": "15px"},
                            ),
                        ],
                        width=2,
                        style={"padding": "0 10px"},
                    ),

                    dbc.Col(
                        [
                            html.Label(
                                "Compare",
                                className="fw-bold",
                                style={"font-size": "16px"}
                            ),
                            dcc.Dropdown(
                                id="countries-multi",
                                options=[{"label": c, "value": c} for c in countries],
                                value=["China", "United States", "India"],
                                multi=True,
                                style={"font-size": "15px"},
                            ),
                        ],
                        width=3,
                        style={"padding": "0 10px"},
                    ),

                    dbc.Col(
                        [
                            html.Label(
                                "Forecast",
                                className="fw-bold",
                                style={"font-size": "16px"}
                            ),
                            dcc.Slider(
                                id="forecast-slider",
                                min=1,
                                max=30,
                                value=10,
                                marks={1: "1y", 10: "10y", 20: "20y", 30: "30y"},
                                step=1,
                            ),
                        ],
                        width=2,
                        style={"padding": "0 10px"},
                    ),

                    dbc.Col(
                        [
                            html.Label(
                                "Target",
                                className="fw-bold",
                                style={"font-size": "16px"}
                            ),
                            dcc.Slider(
                                id="extrapolate-slider",
                                min=2021,
                                max=2050,
                                value=2030,
                                marks={2021: "2021", 2030: "2030", 2050: "2050"},
                                step=1,
                            ),
                        ],
                        width=2,
                        style={"padding": "0 10px"},
                    ),
                ],
                className="g-3",
                align="center",
                style={"width": "100%"},
            ),

            fluid=True,
            style={"max-width": "100%", "padding": "0 20px"},
        ),

        color="light",
        dark=False,
        fixed="top",
        className="shadow-sm border-bottom",
        style={
            "padding": "18px 0",
            "width": "100%",
            "max-width": "100vw",
            "height": "130px",
            "display": "flex",
            "align-items": "center"
        },
    )

    app.layout = dbc.Container([

        navbar_filters,

        html.Div(style={"height": "130px"}),

        dbc.Row([
            dbc.Col(dcc.Graph(id='choropleth-map'), width=6),
            dbc.Col(dcc.Graph(id='top10-bar'), width=6),
        ], className="mb-3"),

        dbc.Row([
            dbc.Col(dcc.Graph(id='heatmap'), width=12),
        ], className="mb-3"),

        dbc.Row([
            dbc.Col(dcc.Graph(id='timeseries-single'), width=6),
            dbc.Col(dcc.Graph(id='timeseries-multi'), width=6),
        ], className="mb-3"),

        dbc.Row([
            dbc.Col(dcc.Graph(id='arima-forecast'), width=6),
            dbc.Col(dcc.Graph(id='linear-trend'), width=6),
        ], className="mb-3"),

    ], fluid=True, style={
        "padding": "20px",
        "max-width": "100%",
        "overflow": "hidden"
    })

   
    # Callback 1: Choropleth map using indicator_4a_emission_share
    @app.callback(
        Output('choropleth-map', 'figure'), 
        Input('year-slider', 'value')
    )
    def update_choropleth(year):
        emissions_year_df = indicator_4a_emission_share(df, year=year, show_top_n=None, show_plot=False)
        year_str = str(year)
        fig = px.choropleth(
            emissions_year_df,
            locations="ISO",
            color=year_str,
            hover_name="Country",
            color_continuous_scale='YlOrRd',
            projection='natural earth',
            labels={year_str: f"Emissions ({year})"},
            title=f"Global Emissions by Country - {year}"
        )
        fig.update_layout(height=450)
        return fig

    # Callback 2: Top N bar chart using indicator_1a_top_emitters_2021
    @app.callback(
        Output('top10-bar', 'figure'), 
        [Input('year-slider', 'value'), Input('top-n-radio', 'value')]
    )
    def update_top10(year, n):
        return indicator_1a_top_emitters_2021(df, year=year, top_n=n, return_figure=True)

    # Callback 3: Stacked bar chart for top countries
    @app.callback(
        Output('heatmap', 'figure'), 
        Input('top-n-radio', 'value')
    )
    def update_heatmap(top_n):
        year_cols = [str(y) for y in range(1990, 2022)]
        available_years = [y for y in year_cols if y in df.columns]
        
        df_temp = df.copy()
        df_temp = df_temp[
            (df_temp["Country"] != "World") &
            (df_temp["Country"] != "European Union (27)")
        ]
        top_countries = df_temp.groupby('Country')[available_years].sum().sum(axis=1).nlargest(top_n).index
        
        df_filtered = df_temp[df_temp['Country'].isin(top_countries)]
        
        bar_data = df_filtered.groupby('Country')[available_years].sum().reset_index()
        bar_data = bar_data.melt(
            id_vars=['Country'], 
            value_vars=available_years, 
            var_name='Year', 
            value_name='Emissions'
        )
        
        fig = px.bar(
            bar_data,
            x='Year',           
            y='Emissions',      
            color='Country',    
            title=f'Emissions Evolution - Top {top_n} Countries (1990-2021)',
            hover_data=['Country', 'Emissions']
        )
        
        fig.update_layout(
            height=500, 
            xaxis_title="Year", 
            yaxis_title="Total Emissions",
            barmode='stack'
        )
        
        return fig

    # Callback 4: Single country time series using indicator_1e_emissions
    @app.callback(
        Output('timeseries-single', 'figure'), 
        Input('country-single', 'value')
    )
    def update_ts_single(country):
        return indicator_1e_emissions(df, country=country, return_figure=True)

    # Callback 5: Multi-country comparison using indicator_1f_countries_comparison
    @app.callback(
        Output('timeseries-multi', 'figure'), 
        Input('countries-multi', 'value')
    )
    def update_ts_multi(countries_list):
        if not countries_list:
            countries_list = ["China", "United States", "India"]
        return indicator_1f_countries_comparison(df, countries=countries_list, return_figure=True)

    # Callback 6: ARIMA forecast using indicator_3a_arima_forecast
    @app.callback(
        Output('arima-forecast', 'figure'), 
        Input('forecast-slider', 'value')
    )
    def update_arima(forecast_years):
        results = indicator_3a_arima_forecast(df, forecast_steps=forecast_years, return_figure=True)
        return results

    # Callback 7: Linear trend using indicator_3b_linear_regression_forecast
    @app.callback(
        Output('linear-trend', 'figure'), 
        Input('extrapolate-slider', 'value')
    )
    def update_linear(extrapolate_to):
        results = indicator_3b_linear_regression_forecast(df, forecast_to_year=extrapolate_to, return_figure=True)
        return results
    
    return app

### Part 6 - Test in Notebook and Launch Dashboard

#### Test in Notebook for printing plot

In [16]:
def main_custom_examples():
    """
    Example function demonstrating custom parameters usage.
    """
    file_path = "historical_emissions.csv"
    df = load_data(file_path)
    
    print("=== Custom Example 1: Top 5 Emitters in 2020 ===")
    indicator_1a_top_emitters_2021(df, year=2020, top_n=5, color="purple")
    
    print("\n=== Custom Example 2: Increase 2000-2021 ===")
    indicator_1b_largest_increase(df, start_year=2000, end_year=2021, top_n=15, color="darkred")
    
    print("\n=== Custom Example 3: Germany Emissions ===")
    indicator_1e_emissions(df, country="Germany", start_year=1995, end_year=2021, color="black")
    
    print("\n=== Custom Example 4: Custom Country Comparison ===")
    indicator_1f_countries_comparison(
        df, 
        countries=["Germany", "Japan", "Brazil", "Australia"], 
        start_year=2000,
        end_year=2021,
        colors=["black", "red", "green", "orange"]
    )
    

    
    print("\n=== Custom Example 5: ARIMA with different order ===")
    indicator_3a_arima_forecast(df, forecast_steps=15, order=(2, 1, 2))
    
    print("\n=== Custom Example 6: Linear regression to 2040 ===")
    indicator_3b_linear_regression_forecast(df, forecast_to_year=2040)
    
    print("\n=== Custom Example 7: Compare emerging economies ===")
    indicator_3c_countries_comparison(
        df,
        countries=["Brazil", "Indonesia", "South Africa", "Mexico"],
        start_year=2000,
        colors=["green", "orange", "purple", "brown"]
    )
    
    print("\n=== Custom Example 8: Using ISO codes ===")
    indicator_3c_countries_comparison(
        df,
        iso_codes=["DEU", "JPN", "GBR", "CAN"],
        use_iso=True,
        start_year=1995
    )
    

    print("\n=== Custom Example 9: Emission share in 2015 ===")
    indicator_4a_emission_share(df, year=2015, color_scale="YlOrRd", show_top_n=15)
    
    print("\n=== Custom Example 10: Trend slope 2000-2021 ===")
    indicator_4b_emission_trend_slope(
        df, 
        start_year=2000, 
        end_year=2021,
        show_top_n=10,
        show_bottom_n=10
    )


#### Launch Dashboard

In [17]:
def main():
    file_path = "historical_emissions.csv"
    df = load_data(file_path)

    app = create_dashboard(df)
    app.run(debug=True, port=8053)

if __name__ == "__main__":
    #main_custom_examples()
    main() 

C:\Users\brunt\AppData\Local\Temp\ipykernel_27860\3528727291.py:37: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

d:\Logiciel\Anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

An unsupported index was provided and will be ignored when e.g. forecasting.

d:\Logiciel\Anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

An unsupported index was provided and will be ignored when e.g. forecasting.

d:\Logiciel\Anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

An unsupported index was provided and will be ignored when e.g. forecasting.

C:\Users\brunt\AppData\Local\Temp\ipykernel_27860\2122488148.py:36: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

d:\Logiciel\Anaconda3\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py: